In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

import gi
gi.require_version("Gtk", "3.0")
gi.require_version("Gdk", "3.0")
from gi.repository import GLib, Gdk, Gtk
gi.require_foreign("cairo")

from math import pi
import random

plt.rcParams["figure.figsize"] = (20, 10)

Napisz prostą grę w odbijanie piłeczki rakietką. W tym zadaniu ważne jest odpowiednie połączenie animacji z obsługą klawiatury, a grafika może być na na poziomie gry „Pong” (patrz Wikipedia). Aby nie męczyć się z oprogramowywaniem ruchów komputerowego przeciwnika gra może polegać na odbijaniu piłki od trzech ścian. Kąt odbicia od ściany nie powinien być dokładnie równy kątowi padania, należy wprowadzać drobne losowe zaburzenia.

In [2]:
class Paddle:
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h

class Okno(Gtk.Window):

    PARTICLE_RADIUS = 8
    MAX_MOVEMENT_PER_FRAME = 4
    
    def __init__(self):
        super(Okno, self).__init__()

        self.paddle = Paddle(50, 50, 15, 75)
        
        # początkowe współrzędne cząsteczki
        self.x = 50
        self.y = 50
        self.w = None
        self.h = None
        
        self.x_movement = 16
        self.y_movement = 4

        self.set_title("Okno GTK")
        self.set_default_size(400, 300)
        self.connect("destroy", Gtk.main_quit)
        self.connect("key-press-event", self.on_key_press)
        self.da = Gtk.DrawingArea()
        self.da.set_size_request(200, 200)
        self.da.connect("draw", self.on_draw)
        self.add(self.da)
        self.show_all()

        # uruchomienie animacji
        self.tid = None
        self.toggle_animation()

    # Aby uruchomić bądź wstrzymać animację trzeba stworzyć bądź usunąć tak
    # zwany "timeout", czyli zapis mówiący pętli zdarzeń GTK że ma obowiązek
    # wywoływać wskazaną procedurę w ustalonych odstępach czasu. Pierwszym
    # argumentem podawanym przy tworzeniu jest odstęp w milisekundach, drugim
    # referencja do procedury, jako trzeci można podać wartość, która będzie
    # przekazywana tej procedurze podczas jej wywoływania (nie jest to nam do
    # niczego potrzebne, więc podajemy None).
    #
    def toggle_animation(self):
        if self.tid is None:
            self.tid = GLib.timeout_add(100, self.on_timeout, None)
        else:
            GLib.source_remove(self.tid)
            self.tid = None

    def on_timeout(self, user_data):
        if self.w is None or self.h is None:
            return False 
        
        if self.x + Okno.PARTICLE_RADIUS < self.w and self.x_movement > 0:
            self.x += self.x_movement
        elif self.x - (Okno.PARTICLE_RADIUS) > self.paddle.x + self.paddle.w and self.x_movement < 0:
            self.x += self.x_movement 
        elif self.x_movement > 0:
            self.x_movement = 0 - self.x_movement
            self.y_movement = self.y_movement + random.randint(0, 4)
        elif self.y > self.paddle.y and self.y < self.paddle.y + self.paddle.h and self.x > self.paddle.x:
            self.x_movement = abs(self.x_movement)
        elif self.x > 0:
            self.x += self.x_movement
        else:
            return False
            
        if self.y + Okno.PARTICLE_RADIUS < self.h and self.y_movement > 0:
            self.y += self.y_movement
        elif self.y - Okno.PARTICLE_RADIUS > 0 and self.y_movement < 0:
            self.y += self.y_movement
        elif self.y_movement > 0:
            self.y_movement = 0 - self.y_movement
        else:
            self.y_movement = abs(self.y_movement)
            
        self.da.queue_draw()
        # zwracana wartość ma znaczenie -- False oznaczałoby chęć zakończenia
        # animacji, timeout zostałby automatycznie usunięty
        return True

    def on_draw(self, widget, ctx):
        w = widget.get_allocated_width()
        h = widget.get_allocated_height()
        self.w = w
        self.h = h

        ctx.arc(self.x, self.y, Okno.PARTICLE_RADIUS, 0.0, 2*pi)
        ctx.close_path()
        ctx.fill()
        
        ctx.move_to(self.paddle.x, self.paddle.y)
        ctx.line_to(self.paddle.x + self.paddle.w, self.paddle.y)
        ctx.line_to(self.paddle.x + self.paddle.w, self.paddle.y + self.paddle.h)
        ctx.line_to(self.paddle.x, self.paddle.y + self.paddle.h)
        ctx.fill()
        

    def on_key_press(self, widget, event):
        w = widget.get_allocated_width()
        h = widget.get_allocated_height()
        
        if event.keyval == Gdk.KEY_Up and self.paddle.y > 0:
            self.paddle.y -= Okno.MAX_MOVEMENT_PER_FRAME
        elif event.keyval == Gdk.KEY_Down and self.paddle.y + self.paddle.h < h:
            self.paddle.y += Okno.MAX_MOVEMENT_PER_FRAME
        
        if event.keyval == Gdk.KEY_space:
            self.toggle_animation()
        elif event.keyval == Gdk.KEY_Escape:
            Gtk.main_quit()
        else:
            return False
        
        self.da.queue_draw()
        
        return True

In [3]:
o = Okno()
Gtk.main()